In [1]:
import bs4 
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import pandas as pd
import re
import time

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.11.1
urllib -> 3.10
pandas -> 1.5.2


In [2]:
#Declaring card variables
cards = []
parking = []
bedroom = []
street = []
neighborhood = []

# https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=1&transacao=venda&onde=,Rio%20de%20Janeiro,Rio%20de%20Janeiro,,,,,city,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro,-22.786856,-43.313089,

#Setting number of pages to be scrapped. Zap Imoveis shwos 359.321 imoveis available for purchase in Rio de Janeiro - RJ. 
# With 30 results being displayed per page, this would result in 11.978 pages to be scrapped. 
# However, the results only show up to page 100.
pages = int(100)

##Going over all website pages
for i in range(pages):
    i = i+1
    ##Getting HTML for that page
    response = 'https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=' + str(i) + '&version=v2'
    print(f"{response}+for page+{str(i)}")
    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

    request = Request(response,headers = headers)

    try:
        response = urlopen(request)
        print(response.status) 
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.status, e.reason) 
    except URLError as e:
        print(e.reason)
          
    #Getting tags
    posts = soup.find('div', {'class': 'listing-wrapper__content'}).findAll('div', class_='result-card')
    
    #Filling in card information using the posts
    for post in posts:
        card = {}
        
        # address
        address = post.find('h2',{'class':'l-text l-u-color-neutral-28 l-text--variant-heading-small l-text--weight-medium card__address'})
        if address == None:
            card['neighborhood'], card['city'] = str('N/A', 'N/A')
        else :
            card['neighborhood'], card['city'], *outro = address.get_text().replace('\n','').replace('  ','').split(',')

        # description
        description = post.find('p', {'class': 'l-text l-u-color-neutral-44 l-text--variant-body-small l-text--weight-regular card__description'})
        if description == None:
            card['description'] = str('N/A')
        else :
            card['description'] = str(description.get_text().replace('-',' ').replace('\n',''))

         # price
        price = post.find('p', class_='l-text l-u-color-neutral-28 l-text--variant-heading-small l-text--weight-bold undefined')
        if price is None:
            card['price'] = 0
        else:
            card['price'] = price.get_text().replace('R$', '').replace('\n', '').replace('.', '')

        #taxes
        iptu = post.find('p',{'class':'l-text l-u-color-neutral-44 l-text--variant-body-small l-text--weight-regular undefined'})
        if iptu == None:
            card['condominio'] = int('0')
            card['iptu'] = int('0')
        else:
            card['condominio'] = int(iptu.get_text().split('|')[0].replace('R$','').replace('\n','').replace('.','').replace(' ','').replace('Cond',''))
            card['iptu'] = int(iptu.get_text().split('|')[1].replace('R$','').replace('\n','').replace('.','').replace(' ','').replace('IPTU',''))

        #area
        area = post.find('p',{'class':'l-text l-u-color-neutral-28 l-text--variant-body-small l-text--weight-regular card__amenity', 'itemprop': 'floorSize'})
        if area == None:
            card['area'] = int(0) 
        else :
            card['area'] = int(area.get_text().replace(' ','').replace('\n','').replace('m²',''))


        #bedrooms
        bedrooms = post.find('p',{'class':'l-text l-u-color-neutral-28 l-text--variant-body-small l-text--weight-regular card__amenity', 'itemprop': 'numberOfRooms'})
        if bedrooms == None:
            card['bedrooms'] = int(1)
        else :
            card['bedrooms'] = int(bedrooms.get_text().replace(' ','').replace('\n',''))

        #suite    
        if re.search("suite|suíte|suites|suítes", card['description'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)  


        #bathrooms
        bathrooms = post.find('p',{'class':'l-text l-u-color-neutral-28 l-text--variant-body-small l-text--weight-regular card__amenity', 'itemprop': 'numberOfBathroomsTotal'})
        if bathrooms == None:
            card['bathrooms'] = int(1)
        else :
            card['bathrooms'] = int(bathrooms.get_text().replace(' ','').replace('\n',''))


        #garage
        try:
            span = post.find("span", {"aria-label": "Quantidade de vagas de garagem"})
            garage = span.find_next_sibling(text=True).strip()
            if garage == None:
                card['garage'] = int(0)
            else:
                card['garage'] = int(garage)
        except:
            card['garage'] = str('N/A')


        #Link
        link = post.find('a', {'class':'result-card'})
        if link == None:
            card['link'] = str('N/A')
        else:
            card['link'] = str(link['href'])


        cards.append(card)

zap_bs4 = pd.DataFrame(cards)
print(zap_bs4.isna().sum())
zap_bs4.head(50)

https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=1&version=v2+for page+1
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=2&version=v2+for page+2
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=3&version=v2+for page+3
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=4&version=v2+for page+4
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=5&version=v2+for page+5
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=6&version=v2+for page+6
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=7&version=v2+for page+7
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=8&version=v2+for page+8
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=9&version=v2+for page+9
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=10&version=v2+for page+10
200
https://www.zapimo

200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=84&version=v2+for page+84
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=85&version=v2+for page+85
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=86&version=v2+for page+86
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=87&version=v2+for page+87
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=88&version=v2+for page+88
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=89&version=v2+for page+89
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=90&version=v2+for page+90
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=91&version=v2+for page+91
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=92&version=v2+for page+92
200
https://www.zapimoveis.com.br/venda/imoveis/rj+rio-de-janeiro/?pagina=93&version=v2+for page+93


,neighborhood,city,description,price,condominio,iptu,area,bedrooms,suite,bathrooms,garage,link
0,Centro,Rio de Janeiro,"UM CHARMOSO EDIFÍCIO HISTÓRICO, COMPLETAMENTE ...",367000,0,0,28,1,0,1,0,https://www.zapimoveis.com.br/lancamento/venda...
1,Centro,Rio de Janeiro,O CENTRO DO RIO ESTÁ SEREQUALIFICANDO. MORAR N...,290000,0,0,32,1,0,1,0,https://www.zapimoveis.com.br/lancamento/venda...
2,Centro,Rio de Janeiro,Economia criativa e inovação. Esses são os pil...,490000,0,0,33,1,0,1,0,https://www.zapimoveis.com.br/lancamento/venda...
3,Copacabana,Rio de Janeiro,CONJUGADÃO REFORMADO EM RUA NOBRE DE COPACABAN...,450000,590,900,30,1,0,1,0,https://www.zapimoveis.com.br/imovel/venda-qui...
4,Ipanema,Rio de Janeiro,Tenho o prazer de apresentar a você este belís...,2850000,2700,650,80,2,1,3,1,https://www.zapimoveis.com.br/imovel/venda-fla...
5,Botafogo,Rio de Janeiro,h2 OPORTUNIDADE DE APARTAMENTO EM VILA RESIDEN...,500000,100,312,40,1,0,1,N/A,https://www.zapimoveis.com.br/imovel/venda-apa...
6,Barra da Tijuca,Rio de Janeiro,"Apartamento no melhor da Barra da Tijuca, a 2 ...",1980000,2000,10000,300,3,1,2,1,https://www.zapimoveis.com.br/imovel/venda-cob...
7,Ipanema,Rio de Janeiro,CORRETOR JORGE CURVELLO CRECI/RJ 070592EXCELEN...,3050000,3700,1113,178,4,1,4,1,https://www.zapimoveis.com.br/imovel/venda-apa...
8,Pechincha,Rio de Janeiro,Oportunidade para realizar um bom negócio e co...,430000,450,1100,97,2,1,2,1,https://www.zapimoveis.com.br/imovel/venda-apa...
9,Jacarepaguá,Rio de Janeiro,Excelente apartamento de 2 quartos no condomín...,425000,700,112,64,2,1,2,1,https://www.zapimoveis.com.br/imovel/venda-apa...


In [4]:
# Checking how does the soup looks like
"""
# Find the desired div elements
listing_content = soup.find('div', {'class': 'listing-wrapper__content'})
result_cards = listing_content.find_all('div', class_='result-card')

# Prettify each result card
prettified_result_cards = [result_card.prettify() for result_card in result_cards]

# Join the prettified result cards into a single string
prettified_html = '\n'.join(prettified_result_cards)

print(prettified_html)
"""

"\n# Find the desired div elements\nlisting_content = soup.find('div', {'class': 'listing-wrapper__content'})\nresult_cards = listing_content.find_all('div', class_='result-card')\n\n# Prettify each result card\nprettified_result_cards = [result_card.prettify() for result_card in result_cards]\n\n# Join the prettified result cards into a single string\nprettified_html = '\n'.join(prettified_result_cards)\n\nprint(prettified_html)\n"

In [9]:
zap_bs4.to_csv('./dataset/zap/zap_bs4.csv', index=False, encoding='utf-8-sig')

In [8]:
zap_bs4.head(100)

,neighborhood,city,description,price,condominio,iptu,area,bedrooms,suite,bathrooms,garage,link
0,Centro,Rio de Janeiro,"UM CHARMOSO EDIFÍCIO HISTÓRICO, COMPLETAMENTE ...",367000,0,0,28,1,0,1,0,https://www.zapimoveis.com.br/lancamento/venda...
1,Centro,Rio de Janeiro,O CENTRO DO RIO ESTÁ SEREQUALIFICANDO. MORAR N...,290000,0,0,32,1,0,1,0,https://www.zapimoveis.com.br/lancamento/venda...
2,Centro,Rio de Janeiro,Economia criativa e inovação. Esses são os pil...,490000,0,0,33,1,0,1,0,https://www.zapimoveis.com.br/lancamento/venda...
3,Copacabana,Rio de Janeiro,CONJUGADÃO REFORMADO EM RUA NOBRE DE COPACABAN...,450000,590,900,30,1,0,1,0,https://www.zapimoveis.com.br/imovel/venda-qui...
4,Ipanema,Rio de Janeiro,Tenho o prazer de apresentar a você este belís...,2850000,2700,650,80,2,1,3,1,https://www.zapimoveis.com.br/imovel/venda-fla...
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Gávea,Rio de Janeiro,Nova Época Vende Cobertura Duplex 4 Quartos co...,5500000,1490,12261,266,4,1,4,2,https://www.zapimoveis.com.br/imovel/venda-cob...
96,Freguesia (Jacarepaguá),Rio de Janeiro,Apartamento no Monte Carlos a venda em Rua Jor...,449380,970,1852,69,3,1,2,1,https://www.zapimoveis.com.br/imovel/venda-apa...
97,Copacabana,Rio de Janeiro,"Localizado na Rua Xavier da Silveira, próximo ...",955000,1400,2559,82,3,0,2,N/A,https://www.zapimoveis.com.br/imovel/venda-apa...
98,Barra da Tijuca,Rio de Janeiro,LOPES ENJOY VENDE : VIA PARQUE SHOPPING!!!Sala...,580000,1100,842,60,0,0,1,1,https://www.zapimoveis.com.br/imovel/venda-loj...
